In [308]:
import pandas as pd
import numpy as np
#from matplotlib import pyplot as plt
import tensorflow as tf
#tf.config.optimizer.set_jit(False)
from tensorflow import keras
from keras.models import load_model
import random
from sklearn.metrics import mean_absolute_error
#import matplotlib
import os

from keras.constraints import Constraint
from keras.layers import Layer
from keras import backend as K

import jax
import jax.nn as nn
from jax.random import normal
from jax.experimental import sparse
#from sparse import bcoo_concatenate
from jax.tree_util import tree_map
import jaxlib
import jax.numpy as jnp
import haiku as hk
import optax
from jax import jit
from functools import partial
import functools
from sklearn.decomposition import TruncatedSVD

from scipy.sparse import csr_matrix
from optax import GradientTransformation

In [235]:
%load_ext autoreload
%autoreload 2

In [81]:
class State_prob_folded(Layer):
    def __init__(self, trainable=False, **kwargs):
        super(State_prob_folded, self).__init__(**kwargs)
        self.supports_masking = True
        self.trainable = trainable
    def build(self, input_shape):
        super(State_prob_folded, self).build(input_shape)
    def call(self, inputs, mask=None):
        return K.pow(K.constant(1.) + K.exp(inputs),-1)
    def get_config(self):
        config = {'trainable': self.trainable}
        base_config = super(State_prob_folded, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    def compute_output_shape(self, input_shape):
        return input_shape

class State_prob_bound(Layer):
    def __init__(self, trainable=False, **kwargs):
        super(State_prob_bound, self).__init__(**kwargs)
        self.supports_masking = True
        self.trainable = trainable
    def build(self, input_shape):
        super(State_prob_bound, self).build(input_shape)
    def call(self, inputs, mask=None):
        return K.pow(K.constant(1.) + K.exp(inputs[0]) * (K.constant(1.)+K.exp(inputs[1])),-1)
    def get_config(self):
        config = {'trainable': self.trainable}
        base_config = super(State_prob_bound, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    def compute_output_shape(self, input_shape):
        return input_shape

class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value
    def __call__(self, w):
        return K.clip(w, self.min_value, self.max_value)
    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}

# Define arguments

In [65]:
data_train = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'
data_valid = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt' 
data_obs = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt' 
o = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/ '
e = 100 
l1_regularization_factor = 0 
l2_regularization_factor = 0 
p = 1000 
num_samples= 128,256,512,1024 
learning_rate= 0.001 
num_resamplings= 10 
num_models= 1 
random_seed= 1


In [89]:
args = {
    "data_train": "'/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'",
    "data_valid": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt",
    "data_obs": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt",
    "output_directory": "/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p/",
    "number_additive_traits": 1,
    "l1_regularization_factor": "0.0001,0.001,0.01,0.1",
    "l2_regularization_factor": "0.0001,0.001,0.01,0.1",
    "num_epochs_grid": 10,
    "num_epochs": 10000,
    "num_samples": "128,256,512,1024",
    "learning_rate": "0.0001,0.001,0.01,0.1",
    "num_resamplings": 10,
    "early_stopping": False,
    "num_models": 10,
    "random_seed": 1
}

data_train_file = args["data_train"]
data_valid_file = args["data_valid"]
data_obs_file = args["data_obs"]
output_directory = args["output_directory"]
number_additive_traits = args["number_additive_traits"]
num_epochs_grid = args["num_epochs_grid"]
num_epochs = args["num_epochs"]
num_resamplings = args["num_resamplings"]
early_stopping = args["early_stopping"]
num_models = args["num_models"]
random_seed = args["random_seed"]

#Grid search arguments
l1 = [float(i) for i in args["l1_regularization_factor"].split(",")]
l2 = [float(i) for i in args["l2_regularization_factor"].split(",")]
batch_size = [int(i) for i in args["num_samples"].split(",")]
learn_rate = [float(i) for i in args["learning_rate"].split(",")]

# Load model data legacy

In [3]:
#Load model data into sparse tensors
def load_model_data(file_dict):
  data_dict = {}
  for name in file_dict.keys():
    #Initialise
    data_dict[name] = {}
    #Column names
    ALL_COLUMNS = list(pd.read_csv(file_dict[name], nrows = 1).columns)
    SELECT_COLUMNS = [i for i in range(len(ALL_COLUMNS)) if str.startswith(ALL_COLUMNS[i], "dataset_")]
    FOLD_COLUMNS = [i for i in range(len(ALL_COLUMNS)) if str.startswith(ALL_COLUMNS[i], "fold_") or ALL_COLUMNS[i]=="WT"]
    BIND_COLUMNS = [i for i in range(len(ALL_COLUMNS)) if str.startswith(ALL_COLUMNS[i], "bind_") or ALL_COLUMNS[i]=="WT"]
    TARGET_COLUMN = [i for i in range(len(ALL_COLUMNS)) if ALL_COLUMNS[i]=="fitness"]
    TARGET_SD_COLUMN = [i for i in range(len(ALL_COLUMNS)) if ALL_COLUMNS[i]=="fitness_sd"]
    SEQUENCE_COLUMN = [i for i in range(len(ALL_COLUMNS)) if ALL_COLUMNS[i]=="variant_sequence"]
    TRAINING_SET_COLUMN = [i for i in range(len(ALL_COLUMNS)) if ALL_COLUMNS[i]=="training_set"]
    
    #Save (sparse) tensors
    data_dict[name]["select"] = tf.convert_to_tensor(np.asarray(pd.read_csv(file_dict[name], usecols = SELECT_COLUMNS)), np.float32)
    data_dict[name]["fold"] = tf.sparse.from_dense(tf.convert_to_tensor(np.asarray(pd.read_csv(file_dict[name], usecols = FOLD_COLUMNS)), np.float32))
    data_dict[name]["bind"] = tf.sparse.from_dense(tf.convert_to_tensor(np.asarray(pd.read_csv(file_dict[name], usecols = BIND_COLUMNS)), np.float32))
    data_dict[name]["target"] = tf.convert_to_tensor(np.asarray(pd.read_csv(file_dict[name], usecols = TARGET_COLUMN)), np.float32)
    data_dict[name]["target_sd"] = tf.convert_to_tensor(np.asarray(pd.read_csv(file_dict[name], usecols = TARGET_SD_COLUMN)), np.float32)
    
    #Save remaining columns
    if len(SEQUENCE_COLUMN)!=0 and len(TRAINING_SET_COLUMN)!=0:
      data_dict[name]["sequence"] = np.asarray(pd.read_csv(file_dict[name], usecols = SEQUENCE_COLUMN))
    if len(TRAINING_SET_COLUMN)!=0:
      data_dict[name]["training_set"] = np.asarray(pd.read_csv(file_dict[name], usecols = TRAINING_SET_COLUMN))
    data_dict[name]["fold_colnames"] = np.asarray([ALL_COLUMNS[i].replace("fold_", "") for i in FOLD_COLUMNS])
    data_dict[name]["bind_colnames"] = np.asarray([ALL_COLUMNS[i].replace("bind_", "") for i in BIND_COLUMNS])
  return data_dict

# Load model data jax

In [349]:
def load_model_data_jax(file_dict):
    data_dict = {}
    for name in file_dict.keys():
        # Initialize
        data_dict[name] = {}

        # Read the entire file once
        df = pd.read_csv(file_dict[name])

        # Column names
        ALL_COLUMNS = list(df.columns)
        SELECT_COLUMNS = [col for col in ALL_COLUMNS if col.startswith("dataset_")]
        FOLD_COLUMNS = [col for col in ALL_COLUMNS if col.startswith("fold_") or col == "WT"]
        BIND_COLUMNS = [col for col in ALL_COLUMNS if col.startswith("bind_") or col == "WT"]
        TARGET_COLUMN = "fitness"
        TARGET_SD_COLUMN = "fitness_sd"
        SEQUENCE_COLUMN = "variant_sequence"
        TRAINING_SET_COLUMN = "training_set"

        # Save (sparse) tensors
        data_dict[name]["select"] = jnp.array(df[SELECT_COLUMNS], dtype=jnp.float32)
        data_dict[name]["fold"] = jnp.array(df[FOLD_COLUMNS], dtype=jnp.float32)
        data_dict[name]["bind"] = jnp.array(df[BIND_COLUMNS], dtype=jnp.float32)
        data_dict[name]["target"] = jnp.array(df[TARGET_COLUMN], dtype=jnp.float32)
        data_dict[name]["target_sd"] = jnp.array(df[TARGET_SD_COLUMN], dtype=jnp.float32)

        # Save remaining columns
        if SEQUENCE_COLUMN in df.columns:
            data_dict[name]["sequence"] = np.array(df[SEQUENCE_COLUMN].values)
        if TRAINING_SET_COLUMN in df.columns:
            data_dict[name]["training_set"] = jnp.expand_dims(jnp.array(df[TRAINING_SET_COLUMN].values), axis=-1)

        data_dict[name]["fold_colnames"] = np.array([col.replace("fold_", "") for col in FOLD_COLUMNS])
        data_dict[name]["bind_colnames"] = np.array([col.replace("bind_", "") for col in BIND_COLUMNS])

    return data_dict

In [350]:
data_train_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_train.txt'

data_valid_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_valid.txt' 

data_obs_file = '/Users/pierredemetz/UCL_work/Crick/doubledeepms/Results//Data/mochi/GRB2-SH3/mochi__fit_tmodel_3state_sparse_dimsum128_subsample100p//dataset_all.txt'

In [351]:
#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)

#Load model data
model_data = load_model_data({
    "train": data_train_file,
    "valid": data_valid_file,
    "obs": data_obs_file
    })

In [352]:
#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)

#Load model data
model_data_jax = load_model_data_jax({
    "train": data_train_file,
    "valid": data_valid_file,
    "obs": data_obs_file
    })

In [353]:
model_data_jax['train']

{'select': DeviceArray([[0., 1.],
              [0., 1.],
              [1., 0.],
              ...,
              [1., 0.],
              [1., 0.],
              [0., 1.]], dtype=float32),
 'fold': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'bind': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'target': DeviceArray([-0.33598602, -0.8518485 , -1.0957463 , ..., -0.26487023,
              -1.166961  , -0.66433465], dtype=float32),
 'target_sd': DeviceArray([0.11158013, 0.172

In [9]:
model_data['train']

{'select': <tf.Tensor: shape=(88171, 2), dtype=float32, numpy=
 array([[0., 1.],
        [0., 1.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [0., 1.]], dtype=float32)>,
 'fold': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x111ad0dc0>,
 'bind': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x292465f10>,
 'target': <tf.Tensor: shape=(88171, 1), dtype=float32, numpy=
 array([[-0.33598602],
        [-0.8518485 ],
        [-1.0957463 ],
        ...,
        [-0.26487023],
        [-1.166961  ],
        [-0.66433465]], dtype=float32)>,
 'target_sd': <tf.Tensor: shape=(88171, 1), dtype=float32, numpy=
 array([[0.11158013],
        [0.17281719],
        [0.09720125],
        ...,
        [0.22753495],
        [0.36643863],
        [0.15336636]], dtype=float32)>,
 'sequence': array([['0000000000000000000000000000000000000000000000000L000000'],
        ['00000000I0000000000000000000000000000000Q000000000000000'],
        ['00000000000

# Resample training data

In [10]:
#Resample training data
def resample_training_data(tensor_dict, n_resamplings, rand_seed):
    
  #Resample observed fitness from error distribution
  np.random.seed(rand_seed)
  observed_fitness = np.array(tensor_dict["target"])
  observed_fitness_sd = np.array(tensor_dict["target_sd"])
    
  observed_fitness_resample = np.array(
    [np.array(
      [observed_fitness[i]+np.random.normal(0, observed_fitness_sd[i]) for i in range(len(observed_fitness))])
    for j in range(n_resamplings)])
  #Save new data
    
  tensor_dict["target"] = tf.expand_dims(tf.convert_to_tensor(np.ravel(observed_fitness_resample), np.float32), -1)
  tensor_dict["select"] = tf.concat([tensor_dict["select"] for i in range(n_resamplings)], axis = 0)
  tensor_dict["fold"] = tf.sparse.concat(axis = 0, sp_inputs=[tensor_dict["fold"] for i in range(n_resamplings)])
  tensor_dict["bind"] = tf.sparse.concat(axis = 0, sp_inputs=[tensor_dict["bind"] for i in range(n_resamplings)])
    
  return(tensor_dict)

# Resample training data jax

In [354]:
def resample_training_data_jax(tensor_dict, n_resamplings, rng):
    # Resample observed fitness from error distribution
    
    observed_fitness = tensor_dict["target"]
    observed_fitness_sd = tensor_dict["target_sd"]
    
    observed_fitness_resample = jnp.array(
    [jnp.array(
      [observed_fitness[i]+(observed_fitness_sd[i] * jax.random.normal(rng, shape=(1,))) for i in range(len(observed_fitness))])
    for j in range(n_resamplings)]
    )
    print('here')
    #Save new data

    tensor_dict["target"] = jax.device_put(jnp.expand_dims(observed_fitness_resample.ravel(), -1))

    select_tensors = [tensor_dict["select"] for i in range(n_resamplings)]  # Assuming n_resamplings is defined
    tensor_dict["select"] = jnp.concatenate(select_tensors, axis=0)

    fold_matrices = [tensor_dict["fold"] for i in range(n_resamplings)]  # Assuming fold tensors are JAX-compatible sparse matrices
    tensor_dict["fold"] = jnp.concatenate(fold_matrices, axis=0)

    bind_matrices = [tensor_dict["bind"] for i in range(n_resamplings)]  # Assuming bind tensors are JAX-compatible sparse matrices
    tensor_dict["bind"] = jnp.concatenate(bind_matrices, axis=0)
    
    return tensor_dict


In [355]:
#Resample training data
num_resamplings = 1

#create the rng
random_seed = 42
rng = jax.random.PRNGKey(random_seed)

if num_resamplings!=0:
    model_data_jax["train"] = resample_training_data_jax(
        tensor_dict = model_data_jax["train"],
        n_resamplings = num_resamplings,
        rng = rng)

here


In [16]:
#Resample training data
num_resamplings = 1

#keep a random seed
random_seed = 42

if num_resamplings!=0:
    model_data["train"] = resample_training_data(
        tensor_dict = model_data["train"],
        n_resamplings = num_resamplings,
        rand_seed = random_seed)

In [356]:
model_data_jax["train"]

{'select': DeviceArray([[0., 1.],
              [0., 1.],
              [1., 0.],
              ...,
              [1., 0.],
              [1., 0.],
              [0., 1.]], dtype=float32),
 'fold': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'bind': DeviceArray([[1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              ...,
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.],
              [1., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'target': DeviceArray([[-0.35659617],
              [-0.88376987],
              [-1.1137005 ],
              ...,
              [-0.3068986 ],
              [-1.2346464 ],
        

In [286]:
model_data["train"]['fold']

# Sparse Layers

In [299]:
sparse_tensor = model_data["train"]['fold']

# Convert the SparseTensor to a dense tensor
dense_tensor = tf.sparse.to_dense(sparse_tensor)

# Print the contents of the dense tensor
print(dense_tensor[0])

test1 = dense_tensor[0].numpy()

print(np.unique(test1, return_counts=True))

tf.Tensor([1. 0. 0. ... 0. 0. 0.], shape=(1057,), dtype=float32)
(array([0., 1.], dtype=float32), array([1055,    2]))


In [311]:
def dim_sparse(data):
    dense_tensor = tf.sparse.to_dense(data)

    # Convert the dense tensor to a Numpy array
    dense_matrix = dense_tensor.numpy()

    # Convert the dense matrix to a Scipy sparse matrix (CSR format)
    sparse_matrix = csr_matrix(dense_matrix)

    # Create and fit a TruncatedSVD instance
    n_components = 50  # Adjust this value based on your desired level of compression
    svd = TruncatedSVD(n_components=n_components)
    reduced_data = svd.fit_transform(sparse_matrix)

    return reduced_data


reduced_data = dim_sparse(model_data['train']['bind'])
# Check the shape of the reduced data
print(reduced_data.shape)

(88171, 50)


In [332]:
dim_sparse(model_data_jax['train']['bind'])

numpy.ndarray

In [301]:
sparse_tensor = model_data["train"]['bind']

# Convert the SparseTensor to a dense tensor
dense_tensor = tf.sparse.to_dense(sparse_tensor)

# Print the contents of the dense tensor
print(dense_tensor)

test1 = dense_tensor[0].numpy()

print(np.unique(test1, return_counts=True))

tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]], shape=(88171, 757), dtype=float32)
(array([0., 1.], dtype=float32), array([755,   2]))


# Shuffle Weights

In [25]:
def shuffle_weights(model, weights=None):
  """Randomly permute the weights in `model`, or the given `weights`.

  This is a fast approximation of re-initializing the weights of a model.

  Assumes weights are distributed independently of the dimensions of the weight tensors
    (i.e., the weights have the same distribution along each dimension).

  :param Model model: Modify the weights of the given model.
  :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
    If `None`, permute the model's current weights.
  """
  if weights is None:
    weights = model.get_weights()
  weights = [np.random.permutation(w.flat).reshape(w.shape) for w in weights]
  # Faster, but less random: only permutes along the first dimension
  # weights = [np.random.permutation(w) for w in weights]
  model.set_weights(weights)

In [70]:
def _shuffle_array(rng, arr):
    flat_arr = arr.ravel()
    shuffled_flat_arr = jax.random.permutation(rng, flat_arr)
    return jnp.reshape(shuffled_flat_arr, arr.shape)

def shuffle_weights(rng, model, weights=None):

    """
    Randomly permute the weights in `model`, or the given `weights`.

    This is a fast approximation of re-initializing the weights of a model.

    Assumes weights are distributed independently of the dimensions of the weight tensors
    (i.e., the weights have the same distribution along each dimension).

    :param Model model: Modify the weights of the given model.
    :param list(ndarray) weights: The model's weights will be replaced by a random permutation of these weights.
    If `None`, permute the model's current weights.
    """

    if weights is None:
        weights = model.params

    rngs = jax.random.split(rng, len(weights))
    shuffled_weights = tree_map(lambda r, w: _shuffle_array(r, w), rngs, weights)

    # Assuming `model` has a method `replace` that replaces its parameters.
    # Adjust this line if the model structure is different.
    new_model = model.replace(params=shuffled_weights)
    return new_model


# Create Model Keras

In [78]:
def create_model(learn_rate, l1, l2, input_dim_select, input_dim_folding, input_dim_binding, number_additive_traits):
  ### INPUT LAYER
  ########################################
  #Input layer select
  input_layer_select = keras.layers.Input(
    shape = input_dim_select,
    name = "input_select")
  #Split
  input_layer_select_folding = keras.layers.Lambda(lambda x: tf.expand_dims(x[:,0],-1))(input_layer_select)
  input_layer_select_binding = keras.layers.Lambda(lambda x: tf.expand_dims(x[:,1],-1))(input_layer_select)
  #Input layer folding
  input_layer_folding = keras.layers.Input(
    shape = input_dim_folding,
    name = "input_fold", sparse=True)
  #Input layer binding
  input_layer_binding = keras.layers.Input(
    shape = input_dim_binding,
    name = "input_bind", sparse=True)
  ### FOLDING LAYERS
  ########################################
  #Folding additive trait layer
  folding_additive_trait_layer = keras.layers.Dense(
    number_additive_traits,
    # input_dim = input_dim,
    kernel_initializer = 'glorot_normal',
    activation = "linear",
    use_bias = False,
    name = "folding_additivetrait")(input_layer_folding)
    # kernel_regularizer = keras.regularizers.l1_l2(l1 = l1, l2 = l2))(input_layerB)
  #Folding nonlinear layer
  folding_nonlinear_layer = State_prob_folded(trainable=False)(folding_additive_trait_layer)
  #Folding additive layer
  folding_additive_layer = keras.layers.Dense(
    1,
    activation = "linear",
    name = "folding_additive",
    kernel_constraint=Between(0, 1e3)#, bias_constraint=Between(-1, 1)
    )(folding_nonlinear_layer)
  ### BINDING LAYERS
  ########################################
  #Binding additive trait layer
  binding_additive_trait_layer = keras.layers.Dense(
    number_additive_traits,
    # input_dim = input_dim,
    kernel_initializer = 'glorot_normal',
    activation = "linear",
    use_bias = False,
    name = "binding_additivetrait",
    kernel_regularizer = keras.regularizers.l1_l2(l1 = l1, l2 = l2))(input_layer_binding)
  #Binding nonlinear layer
  binding_nonlinear_layer = State_prob_bound(trainable=False)([binding_additive_trait_layer, folding_additive_trait_layer])
  #Binding additive layer
  binding_additive_layer = keras.layers.Dense(
    1,
    activation = "linear",
    name = "binding_additive",
    kernel_constraint=Between(0, 1e3)#, bias_constraint=Between(-1, 1)
    )(binding_nonlinear_layer)
  ### OUTPUT LAYERS
  ########################################
  #Multiplicative layer folding
  multiplicative_layer_folding = keras.layers.Multiply()([folding_additive_layer, input_layer_select_folding])
  #Multiplicative layer binding
  multiplicative_layer_binding = keras.layers.Multiply()([binding_additive_layer, input_layer_select_binding])
  #Sum layer
  output_layer = keras.layers.Add()([multiplicative_layer_folding, multiplicative_layer_binding])
  #Create keras model defining input and output layers
  model = keras.Model(
    inputs = [input_layer_select, input_layer_folding, input_layer_binding],
    outputs = [output_layer])
  # Compile model
  opt = keras.optimizers.Adam(learning_rate = learn_rate)
  #Compile the model
  model.compile(
    optimizer = opt,
    loss = 'mean_absolute_error')
  return model

# Grid Search Legacy

In [60]:
def fit_model_grid(param_dict, input_data, n_epochs):
  #Clear session
  keras.backend.clear_session()
  #Summarize results
  print("Grid search using %s" % (param_dict))
  #Set random seeds
  random.seed(random_seed)
  tf.random.set_seed(random_seed)
  #Create model
  model = create_model(
    learn_rate = param_dict['learning_rate'],
    l1=param_dict['l1_regularization_factor'],
    l2=param_dict['l2_regularization_factor'],
    input_dim_select = input_data['train']['select'].shape[1],
    input_dim_folding = input_data['train']['fold'].shape[1],
    input_dim_binding = input_data['train']['bind'].shape[1],
    number_additive_traits = param_dict['number_additive_traits'])
  #Validation data
  validation_data = (
    [input_data['valid']['select'], input_data['valid']['fold'], input_data['valid']['bind']],
    input_data['valid']['target'])
  #Fit the model
  history = model.fit(
    [input_data['train']['select'], input_data['train']['fold'], input_data['train']['bind']],
    input_data['train']['target'],
    validation_data = validation_data,
    epochs = n_epochs,
    batch_size = param_dict['num_samples'],
    shuffle = True,
    verbose = 0,
    use_multiprocessing = True)
  return(history.history["val_loss"][-1])

In [94]:
#######################################################################
## TUNE LEARNING RATE, NUMBER OF SAMPLES AND REGULARISATION PARAMS ##
#######################################################################

if len(l1)==1 and len(l2)==1 and len(batch_size)==1 and len(learn_rate)==1:
  #Only parameters
  num_samples = batch_size[0]
  learning_rate = learn_rate[0]
  l1_regularization_factor = l1[0]
  l2_regularization_factor = l2[0]
else:
  #All combinations of tunable parameters
  parameter_grid = [{
  "num_samples":i,
  "learning_rate":j,
  "l1_regularization_factor":k,
  "l2_regularization_factor":l,
  "number_additive_traits":1} for i in batch_size for j in learn_rate for k in l1 for l in l2]
  #Perform grid search
  grid_results = [fit_model_grid(i, model_data, num_epochs_grid) for i in parameter_grid[:3]]
    
  best_params = parameter_grid[[i for i in range(len(grid_results)) if grid_results[i]==min(grid_results)][0]]
  #Summarize results
  print("Best: %f using %s" % (min(grid_results), best_params))
  #Best parameters
  num_samples = best_params['num_samples']
  learning_rate = best_params['learning_rate']
  l1_regularization_factor = best_params['l1_regularization_factor']
  l2_regularization_factor = best_params['l2_regularization_factor']


256
Grid search using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.0001, 'number_additive_traits': 1}
Grid search using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.001, 'number_additive_traits': 1}
Grid search using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.01, 'number_additive_traits': 1}
Best: 0.401939 using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.0001, 'number_additive_traits': 1}


# Jax Utils

In [263]:
from utils import constrained_gradients, StateProbBound, StateProbFolded, Between, between
from linear import custom_linear

# Create model Jax

In [339]:
from functools import partial
custom_linear_jit = jit(partial(custom_linear, output_size=int(number_additive_traits)))

def create_model_fn(number_additive_traits, l1, l2,rng):
    """
    This function returns a function that creates the model. The model is a
    neural network that predicts the log fold change of a protein. The model
    uses a combination of linear and nonlinear layers. The nonlinear layers
    use additive traits, which are the outputs of the linear layers.
    The outputs of the nonlinear layers are then used to create a multiplicative
    layer that is used in the prediction.
    """
    #def custom_linear_wrapper(inputs, num_traits, rng_key):
        #return custom_linear_jit(inputs, int(num_traits), w_init=None, with_bias=False, rng_key=rng)
    
    def model_fn(inputs_select, inputs_folding, inputs_binding):
        """
        This function creates the model. The model is a neural network that
        predicts the log fold change of a protein. The model uses a combination
        of linear and nonlinear layers. The nonlinear layers use additive traits,
        which are the outputs of the linear layers. The outputs of the nonlinear
        layers are then used to create a multiplicative layer that is used in
        the prediction.
        """
        input_layer_select_folding = jnp.expand_dims(inputs_select[:, 0], -1)
        input_layer_select_binding = jnp.expand_dims(inputs_select[:, 1], -1)

        #inputs_folding = inputs_folding.todense()
        #inputs_binding = inputs_binding.todense()
        
        #folding
        ################################# TESTING IN PROGRESS #########################################
        #linear_sparse = jax.experimental.sparse.sparsify(jax.jit(hk.Linear, static_argnums=0))
        #num = jnp.array(jax.lax.tie_in(inputs_folding, number_additive_traits),dtype=jnp.int32)
        
        #folding_additive_trait_layer = custom_linear_jit(inputs_folding,
        #                                                 w_init=None,
        #                                                 with_bias=False,
        #                                                 rng_key=rng)
        
        #####################################################################################
        inputs_folding = inputs_folding.todense()
        inputs_binding = inputs_binding.todense()
        folding_additive_trait_layer = hk.Linear(number_additive_traits, 
                                                 w_init=hk.initializers.VarianceScaling(1.0, "fan_avg", "truncated_normal"),
                                                 with_bias=False
                                                )(inputs_folding)
        
        folding_nonlinear_layer = StateProbFolded()(folding_additive_trait_layer)
        
        folding_additive_layer = hk.Linear(number_additive_traits,
                                           w_init=hk.initializers.VarianceScaling(1.0, "fan_avg", "truncated_normal"),
                                           with_bias=False#,
                                           #kernel_regularizer=hk.regularizers.L1L2(l1=l1, l2=l2)
                                          )(folding_nonlinear_layer)
        
        #binding
        binding_additive_trait_layer = hk.Linear(number_additive_traits, 
                                                 w_init=hk.initializers.VarianceScaling(1.0, "fan_avg", "truncated_normal"), 
                                                 with_bias=False
                                                )(inputs_binding)
        
        binding_nonlinear_layer = StateProbBound()(binding_additive_trait_layer, folding_additive_trait_layer)
        
        binding_additive_layer = hk.Linear(number_additive_traits,
                                           w_init=hk.initializers.VarianceScaling(1.0, "fan_avg", "truncated_normal"),
                                           with_bias=False
                                          )(binding_nonlinear_layer)

        #output
        multiplicative_layer_folding = folding_additive_layer * input_layer_select_folding
        multiplicative_layer_binding = binding_additive_layer * input_layer_select_binding
        output_layer = multiplicative_layer_folding + multiplicative_layer_binding

        return output_layer

    return model_fn


def create_model_jax(rng, learn_rate, l1, l2, input_dim_select, input_dim_folding, input_dim_binding, number_additive_traits):
    # Create model
    model_fn = create_model_fn(number_additive_traits, l1, l2,rng)
    model = hk.without_apply_rng(hk.transform(model_fn))

    # Create optimizer
    opt = optax.chain(
        optax.adam(learn_rate),
        constrained_gradients(['folding_additive', 'binding_additive'], 0, 1e3),
    )

    # Create regularizer
    #regularizer = hk.regularizers.L1L2(l1=l1, l2=l2)

    return model, opt

# Grid Search Jax

In [340]:
model_data['train']['bind'].shape[1]

757

In [341]:
def generate_batches(input_data, batch_size, rng):
    """Generate batches for training.

    Args:
        input_data: A dictionary of NumPy arrays containing the input data.
        batch_size: The batch size.
        rng: A JAX PRNGKey.

    Yields:
        A tuple of (select, fold, bind, target) batches.
    """
    num_samples = input_data['select'].shape[0]
    indices = jnp.arange(num_samples)

    # Shuffle the training data.
    rng, _ = jax.random.split(rng)
    indices = jax.random.permutation(rng, indices)

    # Generate batches.
    for start_idx in range(0, num_samples, batch_size):
        end_idx = min(start_idx + batch_size, num_samples)
        batch_indices = indices[start_idx:end_idx]

        batch_select = input_data['select'][batch_indices]
        batch_fold = input_data['fold'][batch_indices]
        batch_bind = input_data['bind'][batch_indices]
        batch_target = input_data['target'][batch_indices]

        yield batch_select, batch_fold, batch_bind, batch_target

#Fit model for gridsearch
def fit_model_grid_jax(param_dict, input_data, n_epochs, rng):

    #Summarize results
    print("Grid search using %s" % (param_dict))

    rng_init, rng_batches = jax.random.split(rng)

    #Create model
    model, optimizer = create_model_jax(
        rng = rng_init,
        learn_rate = param_dict['learning_rate'],
        l1=param_dict['l1_regularization_factor'],
        l2=param_dict['l2_regularization_factor'],
        input_dim_select = input_data['train']['select'].shape[1],
        input_dim_folding = input_data['train']['fold'].shape[1],
        input_dim_binding = input_data['train']['bind'].shape[1],
        number_additive_traits = param_dict['number_additive_traits'])

    @jax.jit
    def loss_fn(params, inputs_select, inputs_folding, inputs_binding, target):
        output = model.apply(params, inputs_select, inputs_folding, inputs_binding)
        loss = jnp.mean(jnp.abs(output - target))
        
        # Apply L1 and L2 regularization
        l1_loss = 0
        l2_loss = 0
        for p in jax.tree_util.tree_leaves(params):
            if p.ndim > 1:  # exclude bias parameters
                l1_loss += jnp.sum(jnp.abs(p))
                l2_loss += jnp.sum(jnp.square(p))
        loss = loss + param_dict['l1_regularization_factor'] * l1_loss + param_dict['l2_regularization_factor'] * l2_loss
        
        return loss

    @jax.jit
    def update(params, opt_state, inputs_select, inputs_folding, inputs_binding, target):
        grads = jax.grad(loss_fn)(params, inputs_select, inputs_folding, inputs_binding, target)
        updates, new_opt_state = optimizer.update(grads, opt_state)
        #print('update done')
        new_params = optax.apply_updates(params, updates)
        return new_params, new_opt_state

    params = model.init(rng, input_data['train']['select'], input_data['train']['fold'], input_data['train']['bind'])
    opt_state = optimizer.init(params)

    for epoch in range(n_epochs):
        for batch_data in generate_batches(input_data['train'], param_dict['num_samples'], rng_batches):
            inputs_select, inputs_folding, inputs_binding, target = batch_data
            print(type(inputs_select))
            params, opt_state = update(params, opt_state, inputs_select, inputs_folding, inputs_binding, target)
            print('batch_done')
            
        val_loss = loss_fn(params, input_data['valid']['select'], input_data['valid']['fold'], input_data['valid']['bind'], input_data['valid']['target'])
    return val_loss.item()

In [348]:
#Fit model
if len(l1) == 1 and len(l2) == 1 and len(batch_size) == 1 and len(learn_rate) == 1:
    best_params = {
        "num_samples": batch_size[0],
        "learning_rate": learn_rate[0],
        "l1_regularization_factor": l1[0],
        "l2_regularization_factor": l2[0],
        "number_additive_traits": 1
    }
else:
    parameter_grid = [{
        "num_samples": i,
        "learning_rate": j,
        "l1_regularization_factor": k,
        "l2_regularization_factor": l,
        "number_additive_traits": 1
    } for i in batch_size for j in learn_rate for k in l1 for l in l2]

    rng = jax.random.PRNGKey(random_seed)
    rngs = jax.random.split(rng, len(parameter_grid))
    #print(len(parameter_grid))
    grid_results = [fit_model_grid_jax(params, model_data_jax, num_epochs_grid, rng_key) for params, rng_key in zip(parameter_grid, rngs)]

    best_params = parameter_grid[np.argmin(grid_results)]

    print("Best: %f using %s" % (min(grid_results), best_params))

num_samples = best_params['num_samples']
learning_rate = best_params['learning_rate']
l1_regularization_factor = best_params['l1_regularization_factor']
l2_regularization_factor = best_params['l2_regularization_factor']


Grid search using {'num_samples': 128, 'learning_rate': 0.0001, 'l1_regularization_factor': 0.0001, 'l2_regularization_factor': 0.0001, 'number_additive_traits': 1}


/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:143: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, treedef = jax.tree_flatten(tree)
/Users/pierredemetz/miniconda3/envs/pierre/lib/python3.8/site-packages/haiku/_src/data_structures.py:144: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  return jax.tree_unflatten(treedef, leaves)


<class 'jaxlib.xla_extension.DeviceArray'>
batch_done
<class 'jaxlib.xla_extension.DeviceArray'>
batch_done
<class 'jaxlib.xla_extension.DeviceArray'>
batch_done
<class 'jaxlib.xla_extension.DeviceArray'>
batch_done
<class 'jaxlib.xla_extension.DeviceArray'>
batch_done


KeyboardInterrupt: 